# <font color='blue'>Business Analytics</font>

# <font color='blue'>Capítulo 10 - Financial Analytics</font>
## <font color='blue'>Mini-Projeto 1</font>
### <font color='blue'>AI Bot Trader - Robô Investidor Para Recomendação de Compra e Venda de Criptomoedas</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


![title](imagens/mini-projeto1.png)

## Avisos Antes de Começar

- 1- Nosso objetivo aqui é didático! Não use essa aplicação para fins comerciais sem antes realizar muitos testes.
- 2- A execução do processo de treinamento pode ser bastante demorada dependendo do hardware do seu computador.
- 3- Experimente a aplicação com outras técnicas.
- 4- Este projeto é completo e funcional, mas uma versão mais avançada pode ser encontrada no curso de IA Para Finanças da Formação Engenheiro Blockchain.
- 5- Se executar muitas vezes seguidas este projeto seu acesso à API pode ser bloqueado. Estamos fornecendo o arquivo de dados extraídos no dia da gravação das aulas.

## Definição do Problema e Fonte de Dados

Leia o manual em pdf no Capítulo 10.

## Etapas do Projeto

- Parte 1 - Extração dos Dados em Tempo Real
- Parte 2 - Análise de Dados
- Parte 3 - Construção do Modelo e Otimização Bayesiana
- Parte 4 - Execução do AI Bot Trader
- Parte 5 - Conclusão e Próximos Passos

## Instalando e Carregando os Pacotes

In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
# CryptoCurrency eXchange Trading Library
# https://pypi.org/project/ccxt/
!pip install -q ccxt

In [4]:
# https://pypi.org/project/bayesian-optimization/
!pip install -q bayesian-optimization==1.2

In [5]:
# Imports
import csv
import ccxt
import time
import random
import types
import pkg_resources
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bayes_opt import BayesianOptimization
from pprint import pprint
from datetime import datetime
sns.set()

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Versions" --iversions

Author: Versions

numpy     : 1.21.5
seaborn   : 0.11.2
csv       : 1.0
pandas    : 1.4.4
matplotlib: 3.5.2
ccxt      : 4.1.7



## Parte 1 - Extração dos Dados em Tempo Real

### Função de Gravação dos Dados

In [7]:
# Função para salvar dados em formato csv
def grava_csv(arquivo, dados):
    
    # Abre o arquivo para escrita
    with open(arquivo, mode = 'w') as arquivo_saida:
        
        # Gera o cabeçalho
        arquivo_saida.write("Date,Open,High,Low,Close,Adj Close,Volume\n")
        
        # Grava os dados
        csv_writer = csv.writer(arquivo_saida, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        csv_writer.writerows(dados)

### Função de Conexão à Operadora de Criptomoeda

In [8]:
# Função para fazer conexão à exchange para extração dos dados
# https://www.bitmex.com/
# https://www.bitmex.com/app/apiOverview
def conecta_exchange(exchange, max_retries, symbol, timeframe, since, limit):
    
    # Zera o número de tentativas
    num_retries = 0
    
    # Tenta fazer a conexão
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise

### Funções Para Extração dos Dados

In [9]:
# Função para extração dos dados
def extrai_dados(exchange, max_retries, symbol, timeframe, since, limit):
    
    # Timestamp
    earliest_timestamp = exchange.milliseconds()
    
    # Duração da janela em segundos
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    
    # Duração da janela em milisegundos
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    
    # Diferença de tempo
    timedelta = limit * timeframe_duration_in_ms
    
    # Lista para os dados
    all_ohlcv = []
    
    # Loop
    while True:
        
        # Data de início para extração dos dados
        fetch_since = earliest_timestamp - timedelta
        
        # Conecta na exchange e extrai os dados
        ohlcv = conecta_exchange(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        
        # Se alcançamos o limite, finaliza o loop
        if ohlcv[0][0] >= earliest_timestamp:
            break
        
        # Atualiza o tempo mais cedo
        earliest_timestamp = ohlcv[0][0]
        
        # Atualiza os dados
        all_ohlcv = ohlcv + all_ohlcv
        
        # Print do andamento
        print(len(all_ohlcv), 'registros extraídos de', exchange.iso8601(all_ohlcv[0][0]), 'a', exchange.iso8601(all_ohlcv[-1][0]))
        
        if fetch_since < since:
            break
            
    return all_ohlcv

In [10]:
# Função para extrair os dados e salvar em formato csv
def extrai_dados_para_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    
    # Obtém o id da exchange com o pacote ccxt
    exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True,})
    
    # Checa a consistência 
    if isinstance(since, str):
        since = exchange.parse8601(since)
    
    # Extrai o que está sendo comercializado
    exchange.load_markets()
    
    # Extrai os dados
    ohlcv = extrai_dados(exchange, max_retries, symbol, timeframe, since, limit)
    
    # Contador
    key = 0
    
    # Loop
    for item in ohlcv:
        epoch = int(item[0]) / 1000
        ohlcv[key][0] = datetime.utcfromtimestamp(epoch).strftime('%Y-%m-%d')
        ohlcv[key][5] = int(item[5])
        ohlcv[key].append(ohlcv[key][5])
        ohlcv[key][5] = ohlcv[key][4]
        key += 1
    
    # Comprimento de dados extraídos
    ohlen = len(ohlcv)
    
    # Print do andamento
    pprint("Número de Registros: " + str(ohlen))
    
    # Vamos manter um limite para os dados
    if ohlen > 399:
        ohrem = ohlen - 399
        pprint("Removendo: " + str(ohrem))
        ohlcv = ohlcv[ohrem:]
        
    # Grava os dados em csv
    grava_csv(filename, ohlcv)
    
    # Print
    print('Salvos', len(ohlcv), 'registros no arquivo', filename)

### Parâmetros Para Extração dos Dados

In [11]:
# Define os parâmetros de extração dos dados

# Exchange: https://www.bitmex.com/app/apiOverview
exchange = "bitmex"

# Símbolo da criptomoeda
simbolo = "BTC/USD"

# Janela
janela = "1d"

# Data de início
data_inicio = "2018-01-01T00:00:00Z"

# Arquivo de saída
outfile = "dados/dataset.csv"

### Extração dos Dados

In [12]:
# Executa a extração dos dados
extrai_dados_para_csv(outfile, exchange, 3, simbolo, janela, data_inicio, 100)

TypeError: 'NoneType' object is not subscriptable

## Parte 2 - Análise de Dados

### Carregando e Explorando o Arquivo de Dados

In [ ]:
# Carregando o arquivo do disco
df = pd.read_csv(outfile)

In [ ]:
# Dados
df.head()

In [ ]:
# Shape
df.shape

In [ ]:
# Dados de fechamento
close = df.Close.values.tolist()

In [ ]:
# Outros parâmetros para a versão base do modelo
window_size = 30
skip = 5
l = len(close) - 1

## Parte 3 - Construção do Modelo e Otimização Bayesiana

A parte 3 considera que você tem algum conhecimento em Aprendizado Por Reforço. Esse conhecimento pode ser obtido no curso de Deep Learning II em nível avançado ou em nível básico no curso gratuito de Python Fundamentos aqui na DSA.

### Estratégia de Treinamento

https://arxiv.org/abs/1712.06560

https://openai.com/blog/evolution-strategies/

https://gist.github.com/karpathy/77fbb6a8dac5395f1b73e7a89300318d

http://www.deeplearningbook.com.br/

In [ ]:
# Classe para a estratégia de treinamento
# Usamos Deep Evolution Strategy do OpenAI
class PoliticaTrader:

    # Inputs
    inputs = None

    # Construtor
    def __init__(self, weights, reward_function, population_size, sigma, learning_rate):
        
        # Inicializa os atributos da classe
        self.weights = weights
        self.reward_function = reward_function
        self.population_size = population_size
        self.sigma = sigma
        self.learning_rate = learning_rate

    # Obtém o peso a partir da população
    def get_weights_population(self, weights, population):
        
        # Lista para os pesos
        weights_population = []
        
        # Loop pela população
        for index, i in enumerate(population):
            jittered = self.sigma * i
            weights_population.append(weights[index] + jittered)
        
        return weights_population

    # Obtém os pesos
    def get_weights(self):
        return self.weights

    # Treinamento
    def treinamento(self, epoch = 100, print_every = 1):
        
        # Time
        lasttime = time.time()
        
        # Loop pelas épocas
        for i in range(epoch):
            
            # Lista para a população
            population = []
            
            # Recompensas
            rewards = np.zeros(self.population_size)
            
            # Loop pelo population_size
            for k in range(self.population_size):
                
                x = []
                
                # Loop
                for w in self.weights:
                    x.append(np.random.randn(*w.shape))
                    
                population.append(x)
            
            # Loop
            for k in range(self.population_size):
                
                weights_population = self.get_weights_population(self.weights, population[k])
                rewards[k] = self.reward_function(weights_population)
            
            # Recompensas
            rewards = (rewards - np.mean(rewards)) / np.std(rewards)
            
            # Loop
            for index, w in enumerate(self.weights):
                A = np.array([p[index] for p in population])
                
                # Pesos da rede neural 
                self.weights[index] = (w + self.learning_rate / (self.population_size * self.sigma) * np.dot(A.T, rewards).T)
            
            if (i + 1) % print_every == 0:
                print('Iteração %d. Recompensa: %f' % (i + 1, self.reward_function(self.weights)))
        
        print('Tempo Total de Treinamento:', time.time() - lasttime, 'segundos')

### Arquitetura do Modelo de Rede Neural

In [ ]:
# Classe do Modelo
class Modelo:
    
    # Método construtor
    def __init__(self, input_size, layer_size, output_size):
        
        self.weights = [np.random.randn(input_size, layer_size),
                        np.random.randn(layer_size, output_size),
                        np.random.randn(layer_size, 1),
                        np.random.randn(1, layer_size),]

    # Função para previsão
    def predict(self, inputs):
        
        # Feed forward
        feed = np.dot(inputs, self.weights[0]) + self.weights[-1]
        
        # Decisão de compra (previsão)
        decision = np.dot(feed, self.weights[1])
        
        # Compra (decisão)
        buy = np.dot(feed, self.weights[2])
        
        return decision, buy

    def get_weights(self):
        return self.weights

    def set_weights(self, weights):
        self.weights = weights

### Configuração do AI Bot Trader

In [ ]:
# Função para obter o estado dos dados
def get_state(data, t, n):
    d = t - n + 1
    block = data[d : t + 1] if d >= 0 else -d * [data[0]] + data[0 : t + 1]
    res = []
    for i in range(n - 1):
        res.append(block[i + 1] - block[i])
    return np.array([res])

In [ ]:
# Classe para o agente inteligente (Trader)
class Trader:
    
    # Método construtor
    def __init__(self, population_size, sigma, learning_rate, model, money, max_buy, max_sell, skip, window_size,):
        
        # Inicializa os atributos
        self.window_size = window_size
        self.skip = skip
        self.POPULATION_SIZE = population_size
        self.SIGMA = sigma
        self.LEARNING_RATE = learning_rate
        self.model = model
        self.initial_money = money
        self.max_buy = max_buy
        self.max_sell = max_sell
        self.es = PoliticaTrader(self.model.get_weights(),
                                 self.get_reward,
                                 self.POPULATION_SIZE,
                                 self.SIGMA,
                                 self.LEARNING_RATE,)

    # Método de ação
    def agir(self, sequence):
        decision, buy = self.model.predict(np.array(sequence))
        return np.argmax(decision[0]), int(buy[0])

    # Método para obter recompensa
    def get_reward(self, weights):
        
        # Valor inicial investido
        initial_money = self.initial_money
        starting_money = initial_money
        
        # Pesos
        self.model.weights = weights
        
        # Estado
        state = get_state(close, 0, self.window_size + 1)
        
        # Objetos de controle
        inventory = []
        quantity = 0
        
        # Loop
        for t in range(0, l, self.skip):
            
            # Ação e compra/venda
            action, buy = self.agir(state)
            
            # Próximo estado
            next_state = get_state(close, t + 1, self.window_size + 1)
            
            # Verifica ação e valor inicial investido
            if action == 1 and initial_money >= close[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                    
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
            
            elif action == 2 and len(inventory) > 0:
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                    
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell

            # Próximo estado
            state = next_state
        
        return ((initial_money - starting_money) / starting_money) * 100

    # Treinamento do Trader
    def fit(self, iterations, checkpoint):
        self.es.treinamento(iterations, print_every = checkpoint)

    # Método para recomendação
    def investir(self):
        
        # Valor inicial
        initial_money = self.initial_money
        starting_money = initial_money
        
        # Estado
        state = get_state(close, 0, self.window_size + 1)
        
        # Listas de controle        
        states_sell = []
        states_buy = []
        inventory = []
        quantity = 0
        
        # Loop
        for t in range(0, l, self.skip):
            
            # Ação e compra
            action, buy = self.agir(state)
            
            # Próximo estado
            next_state = get_state(close, t + 1, self.window_size + 1)
            
            # Verifica ação e valor inicial investido
            if action == 1 and initial_money >= close[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
                states_buy.append(t)
                
                print('Dia %d: comprar %d unidades ao preço de %f, saldo total %f' % (t, buy_units, total_buy, initial_money))
            
            elif action == 2 and len(inventory) > 0:
                bought_price = inventory.pop(0)
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                if sell_units < 1:
                    continue
                    
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell
                states_sell.append(t)
                
                try:
                    invest = ((total_sell - bought_price) / bought_price) * 100
                except:
                    invest = 0
                
                print('Dia %d, vender %d unidades ao preço de %f, investimento %f %%, saldo total %f,' % (t, sell_units, total_sell, invest, initial_money))
            
            # Próximo estado
            state = next_state

        # Investimento
        invest = ((initial_money - starting_money) / starting_money) * 100
        
        print('\nGanho Total %f, Valor Total Investido %f' % (initial_money - starting_money, invest))
        
        plt.figure(figsize = (20, 10))
        plt.plot(close, label = 'Valor Real de Fechamento', c = 'g')
        plt.plot(close, 'X', label = 'Previsão de Compra', markevery = states_buy, c = 'b')
        plt.plot(close, 'o', label = 'Previsão de Venda', markevery = states_sell, c = 'r')
        plt.legend()
        plt.show()

### Funções Para Buscar o Melhor Trader

In [ ]:
# Função para encontrar o melhor trader
def melhor_trader(window_size, skip, population_size, sigma, learning_rate, size_network):
    
    # Cria o modelo
    model = Modelo(window_size, size_network, 3)
    
    # Cria o trader
    trader = Trader(population_size, sigma, learning_rate, model, 10000, 5, 5, skip, window_size,)
    
    # Treinamento
    try:
        trader.fit(100, 1000)
        return trader.es.reward_function(trader.es.weights)
    except:
        return 0

In [ ]:
# Função para encontrar o melhor trader de acordo com os hiperparâmetros
def busca_melhor_trader(window_size, skip, population_size, sigma, learning_rate, size_network):
    
    # Variável global
    global accbest
    
    # Hiperparâmetros
    param = {'window_size': int(np.around(window_size)),
             'skip': int(np.around(skip)),
             'population_size': int(np.around(population_size)),
             'sigma': max(min(sigma, 1), 0.0001),
             'learning_rate': max(min(learning_rate, 0.5), 0.000001),
             'size_network': int(np.around(size_network)),}
    
    print('\nBuscando Parâmetros %s' % (param))
    
    # Investimento feito pelo melhor trader
    investment = melhor_trader(**param)
    
    print('Após 100 iterações o investimento foi de %f' % (investment))
        
    return investment

### Otimização Bayesiana Para os Hiperparâmetros do Modelo

In [ ]:
# Modelo para otimização bayesiana de hiperparâmetros
otimizacao_bayesiana = BayesianOptimization(busca_melhor_trader, {'window_size': (2, 50),
                                                                  'skip': (1, 15),
                                                                  'population_size': (1, 50),
                                                                  'sigma': (0.01, 0.99),
                                                                  'learning_rate': (0.000001, 0.49),
                                                                  'size_network': (10, 1000),},)

In [ ]:
%%time
otimizacao_bayesiana.maximize(init_points = 30, n_iter = 50, acq = 'ei', xi = 0.0)

In [ ]:
type(otimizacao_bayesiana)

In [ ]:
# Visualiza o resultado
otimizacao_bayesiana.res

Vamos obter o maior valor para cada hiperparâmetro.

In [ ]:
max([dic['target'] for dic in otimizacao_bayesiana.res])

In [ ]:
[dic['params'] for dic in otimizacao_bayesiana.res]

In [ ]:
max([d['learning_rate'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]])

In [ ]:
max([d['population_size'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]])

In [ ]:
max([d['sigma'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]])

In [ ]:
max([d['size_network'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]])

In [ ]:
max([d['skip'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]])

In [ ]:
max([d['window_size'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]])

## Parte 4 - Execução do AI Bot Trader

### Execução do AI Bot Trader - Modelo Base

In [ ]:
%%time
melhor_trader(window_size = 30, 
              skip = 1, 
              population_size = 15, 
              sigma = 0.1, 
              learning_rate = 0.03, 
              size_network = 500)

In [ ]:
# Cria o modelo
modelo_base = Modelo(input_size = 30, layer_size = 500, output_size = 3)

In [ ]:
# Cria o trader base
trader_base = Trader(population_size = 15, 
                     sigma = 0.1, 
                     learning_rate = 0.03, 
                     model = modelo_base, 
                     money = 10000, 
                     max_buy = 5, 
                     max_sell = 5, 
                     skip = 1, 
                     window_size = 30)

Aqui treinamos o trader!

In [ ]:
%%time
trader_base.fit(500, 100)

In [ ]:
# Recomendações
trader_base.investir()

### Execução do AI Bot Trader - Modelo Otimizado

In [ ]:
%%time
melhor_trader(window_size = int(np.around(max([d['window_size'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]]))), 
              skip = int(np.around(max([d['skip'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]]))), 
              population_size = int(max([d['population_size'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]])), 
              sigma = max([d['sigma'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]]), 
              learning_rate = max([d['learning_rate'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]]), 
              size_network = int(np.around(max([d['size_network'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]]))))

In [ ]:
%%time
modelo_otim = Modelo(input_size = int(np.around(max([d['window_size'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]]))), 
                     layer_size = int(np.around(max([d['size_network'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]]))), 
                     output_size = 3)

In [ ]:
# Cria o trader com otimização
trader_otim = Trader(population_size = int(np.around(max([d['population_size'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]]))), 
                     sigma = max([d['sigma'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]]), 
                     learning_rate = max([d['learning_rate'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]]), 
                     model = modelo_otim, 
                     money = 10000, 
                     max_buy = 5, 
                     max_sell = 5, 
                     skip = int(np.around(max([d['skip'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]]))), 
                     window_size = int(np.around(max([d['window_size'] for d in [dic['params'] for dic in otimizacao_bayesiana.res]]))))

Aqui treinamos o trader!

In [ ]:
%%time
trader_otim.fit(500, 100)

In [ ]:
%%time
trader_otim.investir()

## Parte 5 - Conclusão, Lições Aprendidas e Próximos Passos

- Um projeto como esse é complexo e requer conhecimento avançado.
- Nosso Trader demonstrou bons resultados com baixo volume de dados. Teríamos que validar isso com volumes de dados maiores.
- Diferentes políticas podem ser usadas para treinar o Trader e essa é uma decisão humana.
- Diferentes arquiteturas podem ser usadas no modelo e essa é uma decisão humana.
- Todas as técnicas usadas neste projeto podem ser refinadas com procedimentos mais avançados, tornando o AI Bot Trader ainda mais preciso.

# Fim